In [ ]:
def preprocess_images(img):
    img = tf.cast(img, tf.float32) / tf.reduce_max(img)

    # normalizing image, avoiding zero division
    img /= tf.reduce_max(img)
    img *= 255
    img = tf.where(img <= 30, tf.zeros_like(img), img)


    # increase exposure, contrast streching
    tf.image.adjust_brightness(img, delta=2)
    tf.image.adjust_contrast(img, contrast_factor=2)
    img = (img - tf.reduce_min(img)) / (tf.reduce_max(img) - tf.reduce_min(img))
    # img *= 255

    return img

def preprocess_images_wrapper(images):
    return tf.map_fn(fn=preprocess_images, elems=images)

In [ ]:
from matplotlib import pyplot as plt

test_path = 'D:\\data-science\\brain-tumor-data\\test'
test_files = os.listdir(test_path)

fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(20, 20))

for i, file in enumerate(test_files):
    img = Image.open(test_path + '\\' + file)
    y, x = [int(x[0]) for x in file.split('_')[1:]]

    axes[y, x].imshow(img, cmap='grey', vmin=0, vmax=255)
    axes[y, x].set_axis_off()
    axes[y, 0].set_title('Original')
    axes[y, 1].set_title('Exposure and contrast')
    axes[y, 2].set_title('Background cutoff')
    axes[y, 3].set_title('Color quantization')

plt.tight_layout()

In [ ]:
for _ in range(4):
    for i, mat in enumerate(matrices[100:103]):
        with h5py.File(f"{mat_path}\\{mat}", 'r') as file:
            image = file['cjdata']['image']
            label = int(file['cjdata']['label'][0])

            image = np.array(image) / np.max(image)

            if _ == 1:
                image = exposure.equalize_hist(image)

            if _ == 2:
                image = exposure.equalize_hist(image)
                image = exposure.rescale_intensity(image)

            if _ == 3:
                image[image <= 0.05] = 0
                image = exposure.equalize_hist(image)
                image = exposure.rescale_intensity(image)

            image *= 255
            image = np.array(image).astype(np.uint8)
            img_values = Image.fromarray(image)

            file_name = label_names[label] + f"_{i}_{_}.png"
            img_values.save(data_path_after + '\\test\\' + '\\' + file_name)